In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Process Event objects to find out login/logout users for each associated machine.

### Define functions as designed at the stage of Research, Solution planning

In [2]:
# extract date attribute from event object
# To be used as decorator in list.sort(key)
def get_event_date(event):
    return event.date

# process the list of event object that contains machine, user, date and type
def current_users(events):
    # sort the events by date, important when deciding user login/logout status
    events.sort(key=get_event_date)
    machines = {}  # dictionary to store all the machines from events as keys
    
    for event in events:
        if event.machine not in machines:
            machines[event.machine] = set()   # if encounted first, add the machine:empty_set pair
        if event.type == "login":
            machines[event.machine].add(event.user)  # add 'login' type to the value set of the machine
        elif event.type == "logout" and event.user in machines[event.machine]:
            machines[event.machine].remove(event.user)   # remove 'logout' type from the value set of the machine
    return machines

def generate_report(machines):
    for machine, users in machines.items():
        if len(users) > 0:   # process only the machines with user(s)
            user_list = ", ".join(users)
            print("{}: {}".format(machine, user_list))
    


### Define Event class for simulation

In [3]:
class Event:
  def __init__(self, event_date, event_type, machine_name, user):
    self.date = event_date
    self.type = event_type
    self.machine = machine_name
    self.user = user

### Create sample instances of Event class for testing

In [4]:
events = [
    Event('2020-01-21 12:45:56', 'login', 'myworkstation.local', 'jordan'),
    Event('2020-01-22 15:53:42', 'logout', 'webserver.local', 'jordan'),
    Event('2020-01-21 18:53:21', 'login', 'webserver.local', 'lane'),
    Event('2020-01-22 10:25:34', 'logout', 'myworkstation.local', 'jordan'),
    Event('2020-01-21 08:20:01', 'login', 'webserver.local', 'jordan'),
    Event('2020-01-23 11:24:35', 'login', 'mailserver.local', 'chris'),
    Event('2020-01-24 10:14:15', 'logout', 'webserver.local', 'chris'),
]

### Test

In [5]:
users = current_users(events)
print(users)

{'webserver.local': {'lane'}, 'myworkstation.local': set(), 'mailserver.local': {'chris'}}


In [6]:
generate_report(users)

webserver.local: lane
mailserver.local: chris
